In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [2]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sell_month DESC, name'''

sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sell_month DESC, name


In [3]:
sells_df.dtypes

name           object
sell_year       int64
sell_month      int64
buy_year        int64
buy_month       int64
sell_price    float64
buy_price     float64
diff          float64
qty             int64
sell_amt      float64
buy_amt       float64
gross         float64
pct           float64
profit        float64
kind           object
dtype: object

In [4]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).gross.sum()

sell_year  sell_month  buy_year  buy_month  name 
2021       1           2019      8          PTTEP   -37,200.00
                       2020      4          KKP      27,600.00
                                 8          SENA     26,400.00
                                 12         AMATA     7,800.00
                                            COM7     31,200.00
                                                       ...    
           12          2021      11         POPF     -2,000.00
                                            RCL      11,400.00
                                            SIS      20,250.00
                                 12         IMH       4,800.00
                                            KCE       4,250.00
Name: gross, Length: 178, dtype: float64

In [5]:
sold_grp = sells_df.groupby(['name'])
sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()
sold_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_10832/2967423552.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()


,sell_amt,buy_amt,qty,gross
name,,,,
ADVANC,"113,700.00","114,000.00",600,-300.00
AIMIRT,"123,000.00","120,000.00","10,000","3,000.00"
AMATA,"445,800.00","518,400.00","24,000","-72,600.00"
ASIAN,"693,200.00","453,000.00","38,000","240,200.00"
BCP,"153,000.00","162,000.00","6,000","-9,000.00"
BFIT,"364,000.00","384,000.00","12,000","-20,000.00"
BGC,"332,000.00","300,000.00","30,000","32,000.00"
BKI,"367,000.00","358,500.00","1,300","8,500.00"
CHG,"189,000.00","202,000.00","50,000","-13,000.00"


In [6]:
sold_stocks['sell_price'] = sold_stocks['sell_amt'] / sold_stocks['qty']
sold_stocks['buy_price'] = sold_stocks['buy_amt'] / sold_stocks['qty']
cols = 'sell_amt buy_amt gross qty sell_price buy_price'.split()
sold_stocks[cols].sort_values(['name'],ascending=[True]).style.format(format_dict)

,sell_amt,buy_amt,gross,qty,sell_price,buy_price
name,,,,,,
ADVANC,"113,700.00","114,000.00",-300.00,600,189.50,190.00
AIMIRT,"123,000.00","120,000.00","3,000.00","10,000",12.30,12.00
AMATA,"445,800.00","518,400.00","-72,600.00","24,000",18.57,21.60
ASIAN,"693,200.00","453,000.00","240,200.00","38,000",18.24,11.92
BCP,"153,000.00","162,000.00","-9,000.00","6,000",25.50,27.00
BFIT,"364,000.00","384,000.00","-20,000.00","12,000",30.33,32.00
BGC,"332,000.00","300,000.00","32,000.00","30,000",11.07,10.00
BKI,"367,000.00","358,500.00","8,500.00","1,300",282.31,275.77
CHG,"189,000.00","202,000.00","-13,000.00","50,000",3.78,4.04


### Record selection for active stocks

In [7]:
sql = '''
SELECT name, YEAR(buys.date) AS year, MONTH(buys.date) AS month,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE status = 'Active'
ORDER BY name, buys.date'''
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,year,month,unit_cost,qty,cost_amt
0,BCH,2021,9,21.70,"6,000","130,200.00"
1,BCH,2021,9,21.30,"6,000","127,800.00"
2,BCH,2021,12,20.50,"3,000","61,500.00"
3,BGRIM,2021,9,42.00,"3,000","126,000.00"
4,BGRIM,2021,12,39.50,"3,000","118,500.00"
5,BGRIM,2021,12,38.50,"3,000","115,500.00"
6,CKP,2021,11,5.00,"30,000","150,000.00"
7,DCC,2021,9,2.90,"40,000","116,000.00"
8,DIF,2019,7,14.70,"7,000","102,900.00"
9,DIF,2019,9,14.70,"5,000","73,500.00"


In [8]:
buys_df.groupby(['year','month','name']).cost_amt.sum()

year  month  name 
2016  9      MCS     167,000.00
      10     MCS     334,000.00
      11     MCS     334,000.00
2017  4      MCS     167,000.00
2018  5      JASIF   300,000.00
                        ...    
2021  12     NOBLE   128,100.00
             RATCH   135,750.00
             RCL     103,200.00
             RJH      49,500.00
             STA     147,500.00
Name: cost_amt, Length: 73, dtype: float64

In [9]:
buys_df.groupby(['year','month']).cost_amt.sum()

year  month
2016  9         167,000.00
      10        334,000.00
      11        334,000.00
2017  4         167,000.00
2018  5         300,000.00
2019  7         102,900.00
      9          73,500.00
      11        383,500.00
2020  2         100,000.00
      8         117,600.00
      10        147,000.00
      11        300,000.00
2021  2         147,000.00
      3         826,450.00
      4         165,750.00
      5         198,000.00
      6         587,000.00
      7         525,125.00
      8         654,500.00
      9       1,671,950.00
      10        662,375.00
      11      1,563,050.00
      12      1,420,150.00
Name: cost_amt, dtype: float64

In [10]:
buys_df.cost_amt.sum()

10947850.0

In [11]:
buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_10832/3973236850.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)


,cost_amt,qty
name,,
BCH,"319,500.00","15,000"
BGRIM,"360,000.00","9,000"
CKP,"150,000.00","30,000"
DCC,"116,000.00","40,000"
DIF,"882,000.00","60,000"
DOHOME,"200,000.00","8,000"
EPG,"156,800.00","14,000"
GLOBAL,"150,000.00","7,500"
HREIT,"268,500.00","30,000"


In [12]:
buys_grp = buys_df.groupby(by=['name'])
dtd_stocks = buys_grp['cost_amt','qty'].sum()
dtd_stocks['avg_cost'] = dtd_stocks['cost_amt'] / dtd_stocks['qty']
dtd_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_10832/726847341.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dtd_stocks = buys_grp['cost_amt','qty'].sum()


,cost_amt,qty,avg_cost
name,,,
BCH,"319,500.00","15,000",21.30
BGRIM,"360,000.00","9,000",40.00
CKP,"150,000.00","30,000",5.00
DCC,"116,000.00","40,000",2.90
DIF,"882,000.00","60,000",14.70
DOHOME,"200,000.00","8,000",25.00
EPG,"156,800.00","14,000",11.20
GLOBAL,"150,000.00","7,500",20.00
HREIT,"268,500.00","30,000",8.95


In [13]:
file_name = 'unit-cost.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/unit-cost.csv',
 '\\Users\\User\\iCloudDrive\\unit-cost.csv',
 '\\Users\\User\\Dropbox\\unit-cost.csv')

In [14]:
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(csv_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(box_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(data_file)

### Extra addition

In [15]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sells.date, name'''

print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2021
ORDER BY sells.date, name


In [16]:
df_sells = pd.read_sql(sql, conpf)
df_sells.head()

,name,sell_year,sell_month,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,KKP,2021,1,2021-01-04,4800,50.25,44.50,"240,665.77","214,073.10","26,592.67",12.92,636,DTD
1,SENA,2021,1,2021-01-04,60000,3.38,2.94,"202,350.81","176,790.71","25,560.10",14.97,637,DTD
2,COM7,2021,1,2021-01-19,2600,43.75,38.75,"113,498.06","100,973.15","12,524.91",12.90,638,DTD
3,COM7,2021,1,2021-01-19,2600,45.75,38.75,"118,686.53","100,973.15","17,713.38",18.06,639,DTD
4,AMATA,2021,1,2021-01-27,6000,17.50,16.20,"104,767.44","97,415.28","7,352.16",8.02,640,DTD


In [17]:
ttl_by_month = df_sells.groupby(['sell_month'])['profit'].sum()
ttl_by_month

sell_month
1      51,296.67
2     202,547.56
3     -19,735.49
4      65,267.16
5      79,180.57
6      14,606.32
7    -102,107.17
8      69,193.08
9       1,544.44
10    -10,446.93
11    119,130.67
12     65,786.40
Name: profit, dtype: float64

In [18]:
ttl_by_month = df_sells.groupby(['sell_year','sell_month','name'], as_index=True).agg(
    {
        'profit':['sum','count'],
    }
)
ttl_by_month

profit      
                                   sum count
sell_year sell_month name                   
2021      1          AMATA    7,352.16     1
                     COM7    30,238.29     2
                     KKP     26,592.67     1
                     PTTEP  -38,446.55     1
                     SENA    25,560.10     1
...                                ...   ...
          12         SENA     4,657.77     1
                     SINGER  15,868.48     2
                     SIS     28,437.68     3
                     SYNEX   23,621.26     2
                     VNT      3,742.51     1

[142 rows x 2 columns]

In [19]:
pd.set_option('max_rows',None)
ttl_by_month.get('profit')

sum  count
sell_year sell_month name                      
2021      1          AMATA      7,352.16      1
                     COM7      30,238.29      2
                     KKP       26,592.67      1
                     PTTEP    -38,446.55      1
                     SENA      25,560.10      1
          2          ASIAN     24,457.35      1
                     COM7      20,307.63      1
                     LPF          236.43      1
                     SCCC       6,160.84      1
                     SF         4,545.95      1
                     SIS       24,083.04      1
                     STA       99,978.94      3
                     TTB       22,777.38      1
          3          AMATA    -24,568.76      2
                     ASIAN     31,483.62      3
                     BFIT     -21,656.76      3
                     BGC        1,543.73      1
                     CPTGF        104.63      1
                     FSMART   -45,696.09      4
                     GVREIT       918.91      1
                     LH        -5,077.24      1
                     MC           528.23      1
                     ORI       -1,828.25      3
                     PTG       38,047.15      2
                     SF         4,528.23      1
                     TTLPF        720.09      3
                     UTP          724.23      1
                     WHART        492.79      1
          4          GUNKUL    42,346.40      3
                     ORI       -3,201.56      1
                     RATCH    -11,110.50      2
                     ROJNA     35,901.69      6
                     TTLPF      1,331.13      2
          5          AMATA    -12,583.72      1
                     BKI        7,117.93      1
                     DTAC      -5,805.69      1
                     EGATIF    -1,538.22      1
                     GC        35,548.18      4
                     GUNKUL     7,463.57      1
                     IVL        5,292.90      1
                     MBAX     -22,633.46      1
                     ORI           41.24      1
                     PDG         -431.92      1
                     ROJNA     24,132.60      4
                     TIPH         349.66      1
                     TTLPF     12,946.55      7
                     TU        28,913.18      1
                     WHART        367.77      1
          6          DIF      -92,405.38      4
                     GUNKUL    28,556.76      3
                     MCS      -18,699.91      1
                     ORI       -5,102.46      3
                     POPF         514.94      1
                     ROJNA     12,191.03      2
                     SCC       -1,570.80      2
                     SIS        8,588.87      1
                     SUPEREIF     514.94      1
                     TU        77,183.37      3
                     WHART      4,834.96      2
          7          AIMIRT     2,461.78      1
                     AMATA    -28,460.14      2
                     ASIAN    102,945.71      4
                     CPF      -62,282.43      5
                     DTAC      19,397.54      2
                     EGCO     -80,210.82      3
                     GC         8,587.71      1
                     GUNKUL     9,419.15      1
                     LH       -58,583.44      4
                     ORI       -4,407.54      2
                     TCAP     -13,392.17      2
                     WHART      2,417.48      1
          8          AMATA    -16,475.08      1
                     ASIAN     78,774.63      4
                     BGC       29,056.45      1
                     DCC       32,113.17      4
                     DTAC      10,696.56      1
                     GVREIT       532.66      1
                     JMART     22,334.68      2
                     LH       -14,645.86      1
                     MC           100.21      1
                     MTC       -8,297.37      1
                     ORI       -1,164.75      2
     

In [20]:
ttl_by_month.groupby(level='name').sum()

profit      
                 sum count
name                      
ADVANC       -804.34     1
AIMIRT      2,461.78     1
AMATA     -74,735.54     7
ASIAN     237,661.31    12
BCP        -9,697.69     1
BFIT      -21,656.76     3
BGC        30,600.18     2
BKI         6,893.09     2
CHG       -13,866.04     2
COM7       60,610.68     4
CPF       -62,282.43     5
CPNCG      22,985.57     2
CPTGF         839.50     2
DCC        32,113.17     4
DIF       -92,405.38     4
DTAC       24,288.41     4
EGATIF     -4,365.24     2
EGCO      -80,210.82     3
EPG         3,843.40     1
FSMART    -45,696.09     4
GC         44,135.89     5
GLOBAL         53.81     1
GUNKUL     87,785.88     8
GVREIT      1,451.57     2
HREIT      32,458.43     2
IMH         4,643.18     1
IVL        16,150.01     3
JASIF       1,664.43     3
JMART      22,334.68     2
JMT        18,883.57     1
KCE        50,331.34     4
KKP        21,371.71     2
KTC         3,447.35     3
LH       -122,521.88    10
LPF           236.43     1
MBAX      -22,633.46     1
MC         -2,280.22     3
MCS       -39,395.39     2
MST          -474.44     1
MTC        -8,297.37     1
ORI         4,956.25    16
PDG          -431.92     1
POPF          507.17     3
PRM       -67,797.20     9
PTG        38,047.15     2
PTT         4,390.68     1
PTTEP     -39,447.68     3
PTTGC     -29,301.79     3
RATCH     -11,110.50     2
RCL        10,917.59     2
ROJNA      72,225.32    12
SAT         9,027.39     1
SCC        -1,570.80     2
SCCC        1,218.96     2
SENA       30,217.87     2
SF          9,074.18     2
SGP        -5,267.47     1
SINGER     54,439.32     6
SIS        99,705.63     8
SMPC       -6,001.36     1
SSP         9,296.49     1
STA        99,978.94     3
SUPEREIF    1,003.30     2
SYNEX      30,779.05     3
TCAP      -13,392.17     2
TIPH      -34,591.40     2
TOA          -438.56     1
TPIPP         682.37     8
TQM        -8,077.10     2
TTB        22,777.38     1
TTLPF      14,997.77    12
TU        106,096.55     4
TVO        -6,863.81     1
UTP           724.23     1
VNT         3,742.51     1
WHART       9,826.66     9

In [21]:
ttl_by_month.sum()

profit  sum     536,263.28
        count       254.00
dtype: float64